# Импорт и установка модулей

In [ ]:
%pip install wave
%pip install librosa

In [ ]:
import librosa
import os
import json
import random
from pydub import AudioSegment
import wave
import soundfile as sf

# Создание одной аудиозаписи длинной в час

## Сохраним пути ко всем записям

In [25]:
file_path = './configs/wav/'
paths = []

for _, dirs, files in os.walk(file_path):
    for i in dirs:
        new_path = file_path + i + '/'
        
        for _, dirs2, files2 in os.walk(new_path):
            for l in dirs2:
                new_path2 = new_path + l + '/'
                for _, _, files3 in os.walk(new_path2):
                    for j in files3:
                        paths.append(new_path2 + j)

## Перемешаем их, чтобы попадались разные спикеры

In [14]:
random.shuffle(paths)

## Создаем длинную аудиозапись

In [31]:
common_duration = 0
out_audio = []
iter = 0

while common_duration < 3600.0:
    w = wave.open(paths[iter], 'rb')
    common_duration += w.getnframes() / w.getframerate()
    out_audio.append( [w.getparams(), w.readframes(w.getnframes())] )
    w.close()

    iter += 1


Записываем ее и сохраняем

In [32]:
output = wave.open('hour_far.wav', 'wb')
output.setparams(out_audio[0][0])
for i in range(len(out_audio)):
    output.writeframes(out_audio[i][1])
output.close()

# Разбиваем длинную аудиозапись на короткие в 1с со сдвигом окна в 0.5с

In [33]:
with wave.open('hour_far.wav') as mywav:
    print(mywav.getnframes() / mywav.getframerate())


3623.46725


In [44]:
data, sr = librosa.load('hour_far.wav', sr=22050)

In [45]:
out_data = data[:3600 * 22050]
len(out_data)/22050

3600.0

In [46]:
sf.write('hour_far_3600_22050.wav', out_data, 22050)

In [48]:
data, sr = librosa.load('hour_far_3600_22050.wav', sr=22050)

In [50]:
%mkdir /farph

mkdir: /farph: Read-only file system


In [56]:
for i in range(0, len(data) - 22050, 11025):
    sf.write(f'./farph/hour_far_{i}.wav', data[i: i + 22050], 22050)

In [58]:
file_path = './farph/'
lenght = 0

for _, dirs, files in os.walk(file_path):
    for i in files:
        with wave.open(file_path + i , 'rb') as w:
            lenght += w.getnframes() / w.getframerate()

lenght

7198.0